In [5]:
import fitz

In [6]:
pdf_file_path = "../../datasets/resumes/pdf/0a0fc7e0-ee85-401f-b682-06942d64ac97-Dhruvik's-Resume.pdf";
doc = fitz.open(pdf_file_path)

In [7]:
cv_text = ""
for page in doc:
    cv_text += page.get_text("text")
print(cv_text)

DHRUVIK MALAVIYA
malaviya.dhruvik@gmail.com | +91-9712050513 | 🔗GITHUB | 🔗LINKEDIN
Education
Marwadi University Rajkot, India
2021 - 2024
●
Bachelor of Engineering (B.E) in Computer Engineering | CGPA: 7.58/10
Marwadi University Rajkot, India
2018 - 2021
●
Diploma in Electrical Engineering | CGPA: 9.22/10
Skills
Proficient in Java | javascript | PHP | React.js | Web Development | Node.js |DataStructure and Algorithms
Familiar with Bootstrap | MySQL | Git | MongoDB | Express.js | Laravel | Postman
Work Experience
RedGates It Solutions | php developer intern
Mar,23 - Jun,23
●Working on an ecommerce website which used tech stack as MySQL and Laravel.
●Also managed the admin panel of the website.
Oasis Infobyte | Web development and designing intern
Feb,23 - Mar,23
●Learned HTML,CSS,Javascript while completing tasks in this firm.
●Debugged codes for assigned tasks.
Projects
CodeBoost (🔗Github) (🔗Deployed)
May,23 - July,23
CodeBoost is a fully functional ed-tech platform that enables users 

In [17]:
import spacy
from spacy import displacy

In [21]:
# nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_sm", disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])
nlp_doc = nlp(cv_text)
# for ent in nlp_doc.ents:
#     print(ent.text, ent.label_)
# displacy.serve(nlp_doc, style="ent", port=8080)

/Users/sanughosh/Documents/python-env/myenv3_11/lib/python3.11/site-packages/spacy/displacy/__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:8080 ...

Shutting down server on port 8080.


In [10]:
import re

In [11]:
def get_contact_info(text: str):
    contact_number = None
    pattern = r"\b(?:\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b"
    match = re.search(pattern, text)
    if match:
        contact_number = match.group()
    return contact_number

In [12]:
get_contact_info(cv_text)

'91-9712050513'

In [13]:
def get_email_address(text: str):
    email = None
    pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"
    match = re.search(pattern, text)
    if match:
        email = match.group()
    return email

In [14]:
get_email_address(cv_text)

'malaviya.dhruvik@gmail.com'

In [56]:
def get_name(text: str):
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    
    lines = text.split('\n')
    
    first_3_lines = '|'.join(lines[:1])
    
    name = None
    
    name_pattern = r'^[A-Z][A-Za-z]+(?: [A-Z][A-Za-z]+)+$'
    name_match = re.search(name_pattern, first_3_lines)
    
    if name_match:
        name = name_match.group(0)
        # print("Regex Match -->", name)
    
    if not name:
        name_doc = nlp(first_3_lines)
        for ent in name_doc.ents:
            if ent.label_ == "PERSON":
                name = ent.text
                # print("spaCy Match -->", name)
    
    return name

In [57]:
get_name(cv_text)

First 3 Lines:
 DHRUVIK MALAVIYA
Regex Match --> DHRUVIK MALAVIYA


'DHRUVIK MALAVIYA'

In [58]:
def extract_metadata_info(text: str):
    return {
        "name": get_name(text),
        "phone": get_contact_info(text=text),
        "email": get_email_address(text)
    }

In [60]:
import chromadb
from sentence_transformers import SentenceTransformer
import numpy as np

In [62]:
model = SentenceTransformer('all-MiniLM-L6-v2')

0.01s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [63]:
client = chromadb.Client()

In [65]:
collection = client.create_collection(name="resumes_collection")

In [66]:
def get_embeddings(text: str):
    return model.encode(text).tolist()

In [75]:
def store_data_in_chromadb(resume_text: str, resume_info: dict):
    embeddings = get_embeddings(resume_text)
    name = resume_info.get("name")
    phone = resume_info.get("phone")
    email = resume_info.get("email")

    collection.add(
        ids=[email],
        documents=[resume_text],
        metadatas=[
            {
                "name": name,
                "email": email,
                "phone": phone
            }
        ],
        embeddings=embeddings
    )

In [72]:
def search_resumes(query: str, top_k=2):
    """Search for the most similar resumes based on a query"""
    # Generate the embedding for the query
    query_embedding = get_embeddings(query)
    
    # Perform the vector search in Chroma
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    
    return results

In [73]:
metadata = extract_metadata_info(cv_text)

First 3 Lines:
 DHRUVIK MALAVIYA
Regex Match --> DHRUVIK MALAVIYA


In [76]:
store_data_in_chromadb(resume_text=cv_text, resume_info=metadata)

In [78]:
query = "Java"

In [79]:
search_results = search_resumes(query)

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


In [80]:
print("Search Results:")
for result in search_results['documents']:
    print(result)

Search Results:
['DHRUVIK MALAVIYA\nmalaviya.dhruvik@gmail.com | +91-9712050513 | 🔗GITHUB | 🔗LINKEDIN\nEducation\nMarwadi University Rajkot, India\n2021 - 2024\n●\nBachelor of Engineering (B.E) in Computer Engineering | CGPA: 7.58/10\nMarwadi University Rajkot, India\n2018 - 2021\n●\nDiploma in Electrical Engineering | CGPA: 9.22/10\nSkills\nProficient in Java | javascript | PHP | React.js | Web Development | Node.js |DataStructure and Algorithms\nFamiliar with Bootstrap | MySQL | Git | MongoDB | Express.js | Laravel | Postman\nWork Experience\nRedGates It Solutions | php developer intern\nMar,23 - Jun,23\n●Working on an ecommerce website which used tech stack as MySQL and Laravel.\n●Also managed the admin panel of the website.\nOasis Infobyte | Web development and designing intern\nFeb,23 - Mar,23\n●Learned HTML,CSS,Javascript while completing tasks in this firm.\n●Debugged codes for assigned tasks.\nProjects\nCodeBoost (🔗Github) (🔗Deployed)\nMay,23 - July,23\nCodeBoost is a fully fun

In [81]:
print("Search Results:")
for i, result in enumerate(search_results['documents']):
    # Print document and associated metadata
    print(f"\nResult {i + 1}:")
    print("Document:", result)
    print("Metadata:", search_results['metadatas'][i])

Search Results:

Result 1:
Document: ['DHRUVIK MALAVIYA\nmalaviya.dhruvik@gmail.com | +91-9712050513 | 🔗GITHUB | 🔗LINKEDIN\nEducation\nMarwadi University Rajkot, India\n2021 - 2024\n●\nBachelor of Engineering (B.E) in Computer Engineering | CGPA: 7.58/10\nMarwadi University Rajkot, India\n2018 - 2021\n●\nDiploma in Electrical Engineering | CGPA: 9.22/10\nSkills\nProficient in Java | javascript | PHP | React.js | Web Development | Node.js |DataStructure and Algorithms\nFamiliar with Bootstrap | MySQL | Git | MongoDB | Express.js | Laravel | Postman\nWork Experience\nRedGates It Solutions | php developer intern\nMar,23 - Jun,23\n●Working on an ecommerce website which used tech stack as MySQL and Laravel.\n●Also managed the admin panel of the website.\nOasis Infobyte | Web development and designing intern\nFeb,23 - Mar,23\n●Learned HTML,CSS,Javascript while completing tasks in this firm.\n●Debugged codes for assigned tasks.\nProjects\nCodeBoost (🔗Github) (🔗Deployed)\nMay,23 - July,23\nCod